In [3]:
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import pickle as pkl
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

<IPython.core.display.Javascript object>

In [4]:
plr_names = pd.read_csv('stats.csv')
base_url = 'https://www.mlb.com/player/'

In [5]:
len(plr_names)

101

In [6]:
def get_plr_site_info(i):
    row = plr_names.iloc[i]
    plr_name = f'{row[" first_name"].strip().lower()}-{row["last_name"].lower()}'
    url = f'{base_url}{row[" first_name"].strip().lower()}-{row["last_name"].lower()}-{row["player_id"]}'
    
    r = requests.get(url)
    soup = BS(r.text, 'html.parser')
    return soup, plr_name, row["player_id"]

In [7]:
plr_injury_dates = {}
plr_ids_by_name = {}
for j in range(101):
    soup, plr_name, plr_id = get_plr_site_info(j)
    try:
        table = soup.find(id='transactions').find('table')
    except:
        print(f"Problem with player, {plr_name}")
        break
    tbody = table.find('tbody')
    rows = tbody.find_all('tr')
    table = []
    for row in rows:
        cols = row.find_all('td')
        table.append(cols)
    n_rows = len(table)
    
    injury_dates = []
    for i in range(n_rows):
        entry = table[i][2].string
        if entry is None:
            continue
        try:
            is_injury_row = ('disabled list' in entry or 'injured list' in entry)
        except:
            print(f'Error on row {i} of {plr_name}')
            break
        if is_injury_row:
            placed = 'placed' in entry
            activated = 'activated' in entry
            transferred = 'transferred' in entry
            try:
                assert placed + activated + transferred == 1
            except:
                print(f'Error: the following entry did not follow the placed/activated/transferred trichotomy \n {entry}\n on row {i}')
                break
            if placed:
                injury_dates.append(datetime.strptime(table[i][1].contents[0], '%B %d, %Y'))
    plr_injury_dates[plr_name] = injury_dates
    plr_ids_by_name[plr_name] = plr_id
                
                


In [8]:
injuries_by_name = 'inj_by_name.pkl'
ids_by_name = 'ids_by_name.pkl'
with open(injuries_by_name, 'wb') as f:
    pkl.dump(plr_injury_dates, f)

with open(ids_by_name, 'wb') as f:
    pkl.dump(plr_ids_by_name, f)

In [13]:
list(plr_injury_dates.items())[72]

('kyle-wright', [])

In [2]:
import time
time.sleep(5)
print('hi')

hi
